Notebook de base para armazenar as minhas anotações dos cursos.

Tomarei como base os microdados do enem de 2019. Disponibilizados no [link](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem) . Consultado em 07/01/2021

Por Motivos de Volume de dados e possivel problemas de privacidade eu adicionei a pasta de dados, onde eu vou deixar os dados que vou utilizar nos estudos, no .gitignore desse repositorio. Caso seja do interesse reproduzir o conteudo do que esta ocorrendo aqui, recomendo criar a pasta da mesma forma. 

### Instalação

- A instalação do pyspark varia, dependendo do sistema operacional. Como sempre, recomendo que seja utilizado o linux para a instalação. uma VM já resolve. No meu caso estou rodando o Archlinux

- Para a instalação do Spark, recomendo que seja seguido o tutorial que foi disponibilizado pelo pessoal da sundog Education, para [linux](https://www.tutorialspoint.com/apache_spark/apache_spark_installation.htm), para windows e mac podem ser acompanhado conforme consta no [site](https://sundog-education.com/spark-python/) deles.

- Importante lembrar que, o uso do spark não é recomendado para situações de maquina local, uma vez que, caso seja viavel rodar em um nó unico podemos perfeitamente utilizar o pandas. Eu particularmente gosto de usar o pyspark em uma configuração local para testes menores e preparo para uso em um cluster de produção ou infraestrutura similar.

### importando o pySpark

- Após a instalação, para importar a biblioteca base do pyspark funciona de forma muito simples, com o comando import

``` import pyspark ``` 

- Eu não recomendo que seja utilizado o findmyspark, já que em um ambiente de produção, um AWS EMR por exemplo, não existe a necessidade de ser importado dessa forma, o que pode levar a retrabalhos

In [2]:
import pyspark

### Criando a sessão
- Feito a importação, para executarmos qualquer coisa no spark, precisamos que seja criada uma sessão.
    - A sessão no spark pertence a biblioteca SQL, então para criar precissamos chamar a biblioteca SQL e então o SparkSession.
    - Do SparkSession, precisamos do metodo builder, que funciona para montar a sessão.
    - Após, precisamos do metodo getOrCreate que cria a sessão de fato.
    - A Sessão que criarmos podemos armazenar em uma variavel do python, para podemos executar mais etapas do nosso código.
    - Assim o comando para criarmos a sessão fica:  

In [9]:
sessao = pyspark.sql.SparkSession.builder.getOrCreate()

- Com a Sessão criada, agora podemos executar comandos no spark.

### Importando dados
- Como vamos usar o spark para processos de ETL, vamos precisar fazer a importação de dados. O spark de forma nativa é capaz de ler os principais tipos de dados que são capazes de se comportar bem com volume altos de dados, então CSV, Parquet, avro, json. O uso do jdbc é bem comum para consulta em bancos de dados relacionais. 
- Para importar um csv que esta armazenado de forma local, podemos usar o metodo ```read```  na sessão do spark que criamos previamente. Após o metodo read precisamos informar qual tipo de dado que vamos ler, nesse caso vamos ler um arquivo csv, então usamos a função ```csv``` e apontamos o caminho do arquivo que vamos ler, podendo ele estar dentro de um hdfs, um bucket ou local como é o nosso caso.

In [8]:
sessao.read.csv('dados/MICRODADOS_ENEM_2019.csv')

DataFrame[_c0: string]